In [1]:
import os
import numpy as np
import mss
import cv2
import time
from pynput.keyboard import Key, Controller
import pynput.mouse as pmouse
from collections import deque

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers

Using TensorFlow backend.


In [3]:
def search_screen(template):
    with mss.mss() as sct:
        img = np.array(sct.grab(sct.monitors[0]))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
        w, h = res.shape[::-1]
        idx = np.argmax(res)
        return int(idx%w), int(idx//w)

In [4]:
class Environment:
    def __init__(self, motion=2, delay=25):
        self.action = 0
        self.last_action = 0
        self.actions = ['', Key.up, Key.down]
        self.delay = delay
        self.motion = motion
        self.keyboard = Controller()
        self.mouse = pmouse.Controller()
        self.frames = deque()
        # find browser
        self.gameover_img = cv2.imread('gameover.png', 0)
        gw, gh = self.gameover_img.shape[::-1]
        cord = search_screen(self.gameover_img)
        screen_x = cord[0] - (600 - gw)//2
        screen_y = cord[1] + gh - 2
        self.gameover = {'top': cord[1], 'left': cord[0], 'width': gw, 'height': gh}
        self.monitor = {'top': screen_y, 'left': screen_x, 'width': 600, 'height': 80}
        self.size = (self.monitor['height'] // 10, self.monitor['width'] // 4)
        print('screen', self.monitor)
        print('output', self.size)
        
    def __del__(self):
        self.keyboard.release(Key.up)
        self.keyboard.release(Key.down)
    
    def reset(self):
        self.mouse.position = (self.gameover['left'], self.gameover['top'])
        self.mouse.press(pmouse.Button.left)
        cv2.waitKey(self.delay)
        self.mouse.release(pmouse.Button.left)
        with mss.mss() as sct:
            self.frames.clear()
            for idx in range(self.motion):
                cv2.waitKey(self.delay)
                self.frames.append(self.get_frame(sct))
        return self.get_state()
        
    def get_state(self):
        state = np.zeros((self.motion, self.size[0], self.size[1]))
        ary = []
        for frame in self.frames:
            ary.append(frame)
        return np.dstack(tuple(ary))
        
    def get_frame(self, sct):
        img = np.array(sct.grab(self.monitor))
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        th, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
        img = cv2.resize(img, (0,0), fx=0.25, fy=0.1, interpolation = cv2.INTER_LINEAR)
        return np.array(img, dtype=float) / 255
    
    def is_done(self, sct):
        gameover = cv2.cvtColor(np.array(sct.grab(self.gameover)), cv2.COLOR_BGR2GRAY)
        return np.amax(cv2.matchTemplate(gameover, self.gameover_img, cv2.TM_CCOEFF_NORMED)) > .95
        
    def step(self, verbose=False):
        frame = 0
        done = False
        with mss.mss() as sct:
            while not done and frame < 1999:
                done = self.is_done(sct)
                if not done:
                    if self.action != self.last_action:
                        if self.last_action != 0:
                            self.keyboard.release(self.actions[self.last_action])
                        if self.action != 0:
                            self.keyboard.press(self.actions[self.action])
                        self.last_action = self.action
                    if cv2.waitKey(self.delay) & 0xFF == ord('q'):
                         return
                else:
                    if self.last_action != 0:
                        self.keyboard.release(self.actions[self.last_action])
                reward = 1 if done else 0
                self.frames.pop()
                self.frames.appendleft(self.get_frame(sct))
                frame += 1
                yield self.get_state(), reward
    
    def show(self):
        img = np.array(self.frames[0]*255, dtype=np.uint8)
        img = cv2.resize(img, (0,0), fx=2, fy=5, interpolation = cv2.INTER_NEAREST)
        cv2.imshow('dino', img)

In [8]:
class Agent:
    def __init__(self, env):
        self.env = env
        self.action_size = len(env.actions)
        self.height = env.size[0]
        self.width = env.size[1]
        self.channels = env.motion
        self.shape = [1, self.height, self.width, self.channels]
        self.model = self.get_model(.001)
        self.exploration = .1
        self.decay = .995
        self.memory = deque()
        
    def get_model(self, learning_rate):
        model = Sequential()
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=self.shape[1:]))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(3, activation='linear'))
        #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.compile(loss='mse', optimizer=optimizers.Adam(lr=learning_rate))
        return model
        
    def get_move(self, data, explore=True):
        if explore and np.random.rand() < self.exploration:
            return np.random.randint(self.action_size)
        self.shape[0] = 1
        return np.argmin(self.model.predict(data.reshape(self.shape))[0])
    
    def update(self, gamma):
        self.shape[0] = len(self.memory)
        train_x = np.zeros(self.shape)
        for idx, (state, action, reward) in enumerate(self.memory):
            train_x[idx,:] = state.reshape(self.shape[1:])
        value = 0
        train_y = self.model.predict(train_x)
        for idx, (state, action, reward) in enumerate(self.memory):
            value = reward + value*gamma
            train_y[idx,action] = value
            #train_y[idx,action] += .3*(value - train_y[idx,action])
        self.model.fit(train_x, train_y, batch_size=4, epochs=1, verbose=0)
        self.memory.clear()
        self.exploration = max(self.exploration*self.decay, .01)
        
    def save(self, state, action, reward):
        self.memory.appendleft((state, action, reward))
    
    def save_to_disk(self, filename='trex_model.h5'):
        self.model.save(filename)
        
    def load_from_disk(self, filename='trex_model.h5'):
        if os.path.isfile(filename):
            self.model = keras.models.load_model(filename)

In [11]:
cv2.destroyAllWindows()
env = Environment(2, delay=20)
aj = Agent(env)
aj.load_from_disk()
cv2.imshow('dino', np.zeros((50,50), dtype=np.uint8))
cv2.waitKey(25)

best, episode = 0, 1
running_avg = 0
while True:
    explore = False if episode % 3 == 0 else True
    state = env.reset()
    distance = 0
    for next_state, reward in env.step():
        env.show()
        aj.save(state, env.action, reward)
        state = next_state
        env.action = aj.get_move(state, explore and distance > running_avg)
        distance += 1
    cv2.waitKey(25)
    running_avg += .3*(distance - running_avg)
    if not explore:
        if distance > best + 10:
            aj.save_to_disk('trex_{0}_{1}.h5'.format(episode, distance))
        best = max(best, distance)
    print('\rEpisode: {0}({1}), distance = {2}(best = {3}, running={4}), explore = {5:.2f}% '.format(\
        episode, episode % 3, distance, best, int(running_avg), aj.exploration*100), end='')
    aj.update(.9)
    if distance < 30:
        break
    episode += 1
cv2.destroyAllWindows()
aj.save_to_disk()

screen {'top': 334, 'left': 2056, 'width': 600, 'height': 80}
output (8, 150)
Episode: 50(2), distance = 23(best = 1073, running=457), explore = 7.82%   